In [2]:
import json
from pathlib import Path
from pprint import pprint


file_path='data_medical.json'
data = json.loads(Path(file_path).read_text())

qa_list = []
for d in data:
    statement = "question : {}  answer : {}".format(d['instruction'], d['output'])
    qa_list.append(statement)

In [3]:
import pandas as pd
df = pd.read_json('data_medical.json')
df.rename(columns={'instruction': 'question', 'output':'answer'}, inplace=True)
df.head()

,question,answer
0,전립선은 여성에도 있나요?,아닙니다. 전립선은 남성에게만 있는 생식기관 입니다
1,전립선은 남성에만 있나요?,맞습니다. 전립선은 남성에게만 있는 생식기관 입니다
2,전립선은?,"전립선은 방광 바로 밑, 직장(直腸) 앞쪽에 있는 밤톨만 한 크기의 남성 생식기관으..."
3,전립선이 무엇인가요?,"정상 성인의 전립선은 20g 가량의 호두알 정도 크기의 기관으로 방광의 아래쪽, 직..."
4,전립선이란?,전립선은 방광과 요도 사이에 위치하는 밤알 정도 크기의 생식샘으로 정액에 일부 성분...


In [4]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
OPENAI_TEMPERATURE = float(os.getenv("OPENAI_TEMPERATURE", 0.0))


openai.api_key = OPENAI_API_KEY
print(OPENAI_API_KEY)

In [5]:
df = df.dropna(how='any', axis= 0)

In [7]:
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

def get_embedding(text, model="text-embedding-ada-002"):
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

df['question_vector'] = df.question.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
df['answer_vector'] = df.answer.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
df.head()

0it [00:00, ?it/s]


,question,answer,question_vector,answer_vector
0,전립선은 여성에도 있나요?,아닙니다. 전립선은 남성에게만 있는 생식기관 입니다,"[-0.012017258442938328, -0.03106275387108326, ...","[-0.00876571238040924, -0.01925969123840332, 0..."
1,전립선은 남성에만 있나요?,맞습니다. 전립선은 남성에게만 있는 생식기관 입니다,"[-0.007469899486750364, -0.027836550027132034,...","[-0.00971006229519844, -0.02496306225657463, 0..."
2,전립선은?,"전립선은 방광 바로 밑, 직장(直腸) 앞쪽에 있는 밤톨만 한 크기의 남성 생식기관으...","[0.005338818766176701, -0.026521461084485054, ...","[-0.006054476369172335, -0.010293284431099892,..."
3,전립선이 무엇인가요?,"정상 성인의 전립선은 20g 가량의 호두알 정도 크기의 기관으로 방광의 아래쪽, 직...","[0.004732365719974041, -0.01961228810250759, 0...","[-0.004071803297847509, -0.0005756918690167367..."
4,전립선이란?,전립선은 방광과 요도 사이에 위치하는 밤알 정도 크기의 생식샘으로 정액에 일부 성분...,"[-0.0030415847431868315, -0.019522881135344505...","[0.011263073422014713, -0.003701198846101761, ..."


In [14]:
df.head()

,question,answer,question_vector,answer_vector
0,전립선은 여성에도 있나요?,아닙니다. 전립선은 남성에게만 있는 생식기관 입니다,"[-0.012017258442938328, -0.03106275387108326, ...","[-0.00876571238040924, -0.01925969123840332, 0..."
1,전립선은 남성에만 있나요?,맞습니다. 전립선은 남성에게만 있는 생식기관 입니다,"[-0.007469899486750364, -0.027836550027132034,...","[-0.00971006229519844, -0.02496306225657463, 0..."
2,전립선은?,"전립선은 방광 바로 밑, 직장(直腸) 앞쪽에 있는 밤톨만 한 크기의 남성 생식기관으...","[0.005338818766176701, -0.026521461084485054, ...","[-0.006054476369172335, -0.010293284431099892,..."
3,전립선이 무엇인가요?,"정상 성인의 전립선은 20g 가량의 호두알 정도 크기의 기관으로 방광의 아래쪽, 직...","[0.004732365719974041, -0.01961228810250759, 0...","[-0.004071803297847509, -0.0005756918690167367..."
4,전립선이란?,전립선은 방광과 요도 사이에 위치하는 밤알 정도 크기의 생식샘으로 정액에 일부 성분...,"[-0.0030415847431868315, -0.019522881135344505...","[0.011263073422014713, -0.003701198846101761, ..."


In [250]:
result = df.to_json(force_ascii=False, orient = 'records')

In [251]:
output = json.loads(result)

In [252]:
with open('./data_medical_embedding2.json', 'w', encoding='utf-8') as outfile:
    json.dump(output, outfile , ensure_ascii=False, indent=4)

In [74]:
import chromadb

chroma_client = chromadb.Client()

In [75]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.

# Note. alternatively you can set a temporary env variable like this:
# os.environ["OPENAI_API_KEY"] = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")


embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'), model_name='text-embedding-ada-002')

medical_question_collection = chroma_client.create_collection(name='medical_question', embedding_function=embedding_function)
medical_answer_collection = chroma_client.create_collection(name='medical_answer', embedding_function=embedding_function)

OPENAI_API_KEY is ready


In [105]:
df = df.astype({'vector_id':'str'})

In [122]:
batch_size = 166
chunks = [df[i:i + batch_size] for i in range(0, len(df), batch_size)]

# Add the content vectors in batches
for chunk in chunks:
    medical_question_collection.add(
        ids=chunk['vector_id'].tolist(),
        embeddings=chunk['question_vector'].tolist(),  # Assuming you have the 'question_vector' column
    )

# Assuming you have your medical_answer_collection and answer_vector ready

# Add the title vectors in batches
for chunk in chunks:
    medical_answer_collection.add(
        ids=chunk['vector_id'].tolist(),
        embeddings=chunk['answer_vector'].tolist(),  # Assuming you have the 'answer_vector' column
    )

In [209]:
def query_collection(collection, query, max_results, dataframe):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances']) 
    df = pd.DataFrame({
                'id':results['ids'][0], 
                'score':results['distances'][0],
                'question': dataframe[dataframe.vector_id.isin(results['ids'][0])]['question'],
                'answer': dataframe[dataframe.vector_id.isin(results['ids'][0])]['answer'],
                })
    return df

In [219]:
LLM_MODEL = os.getenv("LLM_MODEL", os.getenv("OPENAI_API_MODEL", "gpt-3.5-turbo")).lower()
OPENAI_TEMPERATURE = float(os.getenv("OPENAI_TEMPERATURE", 0.0))


In [244]:
def openai_call(
    system_prompt: str,
    prompt: str,
    model: str = LLM_MODEL,
    temperature: float = OPENAI_TEMPERATURE,
    max_tokens: int = 100,
):
    while True:
        try:
            if not model.lower().startswith("gpt-"):
                # Use completion API
                response = openai.Completion.create(
                    engine=model,
                    prompt=prompt,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=1,
                    frequency_penalty=0,
                    presence_penalty=0,
                )
                return response.choices[0].text.strip()
            else:
                # Use 4000 instead of the real limit (4097) to give a bit of wiggle room for the encoding of roles.
                # TODO: different limits for different models.

                trimmed_prompt = limit_tokens_from_string(prompt, model, 4000 - max_tokens)

                # Use chat completion API
                messages = [{"role": "system" , "content": system_prompt},{"role": "system", "content": trimmed_prompt}]
                response = openai.ChatCompletion.create(
                    model='gpt-3.5-turbo',
                    messages=messages,
                    temperature=temperature,
                    max_tokens=2000,
                    n=1,
                    stop=None,
                )    
                return response.choices[0].message.content.strip()
        except openai.error.RateLimitError:
            print(
                "   *** The OpenAI API rate limit has been exceeded. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.Timeout:
            print(
                "   *** OpenAI API timeout occurred. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.APIError:
            print(
                "   *** OpenAI API error occurred. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.APIConnectionError:
            print(
                "   *** OpenAI API connection error occurred. Check your network settings, proxy configuration, SSL certificates, or firewall rules. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.InvalidRequestError:
            print(
                "   *** OpenAI API invalid request. Check the documentation for the specific API method you are calling and make sure you are sending valid and complete parameters. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        except openai.error.ServiceUnavailableError:
            print(
                "   *** OpenAI API service unavailable. Waiting 10 seconds and trying again. ***"
            )
            time.sleep(10)  # Wait 10 seconds and try again
        else:
            break

In [201]:
import tiktoken as tiktoken
import time


def limit_tokens_from_string(string: str, model: str, limit: int) -> str:
    """Limits the string to a number of tokens (estimated)."""

    try:
        encoding = tiktoken.encoding_for_model(model)
    except:
        encoding = tiktoken.encoding_for_model('gpt2')  # Fallback for others.

    encoded = encoding.encode(string)

    return encoding.decode(encoded[:limit])

In [257]:
question = '국소전립선암에서 방사선 치료의 역할은?'

In [258]:
title_query_result = query_collection(
    collection=medical_question_collection,
    query=question,
    max_results=3,
    dataframe=df
)
qa_list= [ ]
qa_set = ''
question_list = title_query_result.question
answer_list = title_query_result.answer
for q, a  in zip(question_list,answer_list):
    qa_set += "question: {} answer: {} /n ".format(q, a)
    # qa_list.append("question: {} answer: {}".format(q, a))
print(qa_set)

question: 국소전립선암에서 방사선 치료의 역할은?  answer: 방사선 치료 또한 전립선암을 완치하기 위한 일차 치료법입니다. 방사선 치료의 목적은 전립선에 방사선을 조사하여 세포손상을 유발하여 전립선 내에 있는 암세포를 죽이는 것입니다. 치료 기술의 발전에 힘입어 방사선 치료는 현재 국소 전립선암에 대한 효과적인 치료 방법으로 자리잡고 있습니다. /n question: 국소전립선암의 방사선 치료는 어떠한 방법이 있는지요? answer: 방사선 치료는 외부방사선치료 (external beam radiation therapy; EBRT) 혹은 근접방사선치료 (brachytherapy)로 시행될 수 있습니다. 치료 기술의 발전에 힘입어 방사선 치료는 현재 국소 전립선암에 대한 효과적인 치료 방법으로 자리잡고 있습니다. /n question: 국소전립선암에서 근치적 전립선적출술의 역할은 ?  answer: 현재까지 국소 전립선암에서 보존적 치료와 비교하여 전향적, 무작위 배정 연구에서 유일하게 암으로 사망할 확률에 낮춘 명백한 증거를 보인 치료법은 수술 방법에 상관없이 근치적 전립선적출술 뿐입니다. 또한 아직 어떠한 비수술적 치료에서도 수술적 치료 보다 높은 치료 성적을 보인 명백한 증거는 없습니다. /n 


In [249]:
import openai
system_prompt = "you are an AI Docter"
prompt = f"""
        Provide an appropriate answer to the question based on the given Question-Answer set \n

        Question: 
            `
                {question}
            `

        Qusetion-Answer Set
            `   
                {qa_set}
            `
        """
response = openai_call(system_prompt, prompt, max_tokens=2000)
print(response)

적극적 관찰을 시행한 경우 예후(생존률) 등에 대한 장기 추적 관찰 결과나 표준 치료법과의 무작위 비교 연구 결과가 없습니다. 따라서, 적극적 관찰은 매우 진행의 위험도가 낮은 일부 환자에서 시험적으로 시행되며, 주기적인 전립선특이항원 수치 검사와 전립선 조직검사를 받아야 하는 불편함과 불안감을 동반합니다.
